<a href="https://colab.research.google.com/github/hyeonsang010716/Recommendation-system/blob/main/collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-25m.zip
!unzip ml-25m.zip

--2024-03-28 04:53:29--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip          100%[===================>] 249.84M  78.2MB/s    in 3.2s    

2024-03-28 04:53:32 (78.2 MB/s) - ‘ml-25m.zip’ saved [261978986/261978986]

Archive:  ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


In [ ]:
import numpy as np
users = []
items = []
ratings = []

with open("ml-25m/ratings.csv") as f:
  print(f.readline())
  for line in f:
    uid , mid , rating , _ = line.split(",")
    users.append(int(uid))
    items.append(int(mid))
    ratings.append(float(rating))
users = np.array(users)
items = np.array(items)
ratings = np.array(ratings)

userId,movieId,rating,timestamp



In [ ]:
#기본 RMSE

m = ratings.mean()
RMSE = ((ratings - m) ** 2).mean() ** 0.5
RMSE

1.0607439399275531

In [ ]:
user_bias = np.zeros(users.max() + 1)
item_bias = np.zeros(items.max() + 1)
alpha = ratings.mean()



In [ ]:
lr = 1
lmd = 0.001

n_ratings = len(ratings)
n_users = len(user_bias)
n_items = len(item_bias)

for epoch in range(20):
  h = alpha + user_bias[users] + item_bias[items]
  diff = h - ratings
  rmse = (diff ** 2).mean() ** 0.5
  print("epoch : {} , rmse : {}".format(epoch , rmse))
  grd_alpha = diff.mean()
  grd_user_bias = np.bincount(users , weights = diff) / n_ratings + lmd * user_bias
  grd_item_bias = np.bincount(items , weights = diff) / n_ratings + lmd * item_bias
  alpha = alpha - lr * grd_alpha
  user_bias = user_bias - lr * grd_user_bias
  item_bias = item_bias - lr * grd_item_bias

rmse = (diff ** 2).mean() ** 0.5
print("epoch : {} , rmse : {}".format(epoch , rmse))


epoch : 0 , rmse : 1.0607439399275531
epoch : 1 , rmse : 1.0606227041512328
epoch : 2 , rmse : 1.0605019292889002
epoch : 3 , rmse : 1.060381604488675
epoch : 4 , rmse : 1.0602617275043111
epoch : 5 , rmse : 1.0601422961027653
epoch : 6 , rmse : 1.0600233080641188
epoch : 7 , rmse : 1.0599047611814914
epoch : 8 , rmse : 1.0597866532609672
epoch : 9 , rmse : 1.059668982121504
epoch : 10 , rmse : 1.0595517455948595
epoch : 11 , rmse : 1.0594349415254984
epoch : 12 , rmse : 1.0593185677705277
epoch : 13 , rmse : 1.0592026221996016
epoch : 14 , rmse : 1.0590871026948503
epoch : 15 , rmse : 1.058972007150802
epoch : 16 , rmse : 1.0588573334742943
epoch : 17 , rmse : 1.0587430795844113
epoch : 18 , rmse : 1.0586292434123923
epoch : 19 , rmse : 1.0585158229015668
epoch : 19 , rmse : 1.0585158229015668


In [ ]:
import torch

ratings_tensor = torch.from_numpy(ratings)
alpha = torch.tensor(ratings.mean() , requires_grad = True)
user_bias = torch.zeros(users.max() + 1 , requires_grad = True)
item_bias = torch.zeros(items.max() + 1 , requires_grad = True)
lmd = 0.001
optim = torch.optim.SGD([alpha , user_bias , item_bias] , lr = 1 )

for epoch in range(20):
  h = alpha + user_bias[users] + item_bias[items]
  mse = ((h - ratings_tensor) ** 2).mean()
  reg = lmd * (((user_bias) ** 2).mean() + (item_bias ** 2).mean())
  cost = mse + reg
  optim.zero_grad()
  cost.backward()
  optim.step()

  with torch.no_grad():
    rmse = ((h - ratings_tensor) ** 2).mean() ** 0.5
    print("epoch : {} , rmse : {} ".format(epoch , rmse))


epoch : 0 , rmse : 1.0607439399275533 
epoch : 1 , rmse : 1.0605016242212497 
epoch : 2 , rmse : 1.0602606413495452 
epoch : 3 , rmse : 1.0600209894996253 
epoch : 4 , rmse : 1.05978263276361 
epoch : 5 , rmse : 1.059545571873008 
epoch : 6 , rmse : 1.0593097959363704 
epoch : 7 , rmse : 1.0590752792200044 
epoch : 8 , rmse : 1.0588420273709978 
epoch : 9 , rmse : 1.058610017432027 
epoch : 10 , rmse : 1.0583792505466205 
epoch : 11 , rmse : 1.058149716351938 
epoch : 12 , rmse : 1.0579214030635105 
epoch : 13 , rmse : 1.0576943162900916 
epoch : 14 , rmse : 1.057468431092836 
epoch : 15 , rmse : 1.057243734949861 
epoch : 16 , rmse : 1.0570202218734992 
epoch : 17 , rmse : 1.0567978761373642 
epoch : 18 , rmse : 1.0565766983487688 
epoch : 19 , rmse : 1.0563566787566905 


In [ ]:
lmd = 0.001

user_bias = np.zeros(users.max() + 1)
item_bias = np.zeros(items.max() + 1)
alpha = ratings.mean()
for epoch in range(10):
  h = alpha + user_bias[users] + item_bias[items]
  rmse = ((h - ratings) ** 2).mean() ** 0.5
  print("epoch : {} , rmse : {}".format(epoch , rmse))
  alpha = (ratings - (user_bias[users] + item_bias[items])).mean()
  user_bias = np.bincount(users , weights = ratings - (alpha + item_bias[items])) / (np.bincount(users) + lmd)
  item_bias = np.bincount(items , weights = ratings - (alpha + user_bias[users])) / (np.bincount(items) + lmd)

print("final rmse : {}".format(rmse))

epoch : 0 , rmse : 1.0607439399275531
epoch : 1 , rmse : 0.8663159834756426
epoch : 2 , rmse : 0.8510867185839471
epoch : 3 , rmse : 0.8503568292676987
epoch : 4 , rmse : 0.8503078899843288
epoch : 5 , rmse : 0.8503025377563744
epoch : 6 , rmse : 0.8503016450633218
epoch : 7 , rmse : 0.8503014646716495
epoch : 8 , rmse : 0.8503014258804191
epoch : 9 , rmse : 0.8503014173907169
final rmse : 0.8503014173907169
